In [102]:
import pandas as pd


### Загрузка данных

In [103]:
project_1_data = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')   # импорт из файла
hh_data = project_1_data.copy()                                              # создаём копию для работы
display(hh_data.head(2))
display(hh_data.info())

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Пол, возраст                     44744 non-null  object
 1   ЗП                               44744 non-null  object
 2   Ищет работу на должность:        44744 non-null  object
 3   Город, переезд, командировки     44744 non-null  object
 4   Занятость                        44744 non-null  object
 5   График                           44744 non-null  object
 6   Опыт работы                      44576 non-null  object
 7   Последнее/нынешнее место работы  44743 non-null  object
 8   Последняя/нынешняя должность     44742 non-null  object
 9   Образование и ВУЗ                44744 non-null  object
 10  Обновление резюме                44744 non-null  object
 11  Авто                             44744 non-null  object
dtypes: object(12)
memory usage: 4.1+

None

### Анализ столбца пол/возраст/др

In [104]:
hh_data_dr = hh_data[['Пол, возраст']].copy()
#display(hh_data_dr.info())

def f_razdelenie_stroki(stroka, razdelitel=','):   # разбиение по разделителю и удаление лишних пробелов
    spisok_stroka = stroka.lower()
    spisok_stroka = spisok_stroka.split(razdelitel)
    for n in range(0, len(spisok_stroka)):
        spisok_stroka[n] = spisok_stroka[n].strip()
    return spisok_stroka

def f_pol(stroka, razdelitel=','):                      # функция выделения данных пола
    pol = f_razdelenie_stroki(stroka, razdelitel)[0]
    if pol == 'мужчина' or 'женщина':
        return pol
    return 0

def f_vozrast(stroka, razdelitel=','):                   # функция выделения данных возраста
    vozrast = f_razdelenie_stroki(stroka, razdelitel)[1]
    spisok_vozrast = vozrast.split(' ')
    if len(spisok_vozrast) == 2:
        return spisok_vozrast[0]
    return 0

def f_dr(stroka, razdelitel=','):                        # функция выделения данных ДР
    dr = f_razdelenie_stroki(stroka, razdelitel)[2]
    spisok_pol_dr = dr.split(' ')
    month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
           'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
    if len(spisok_pol_dr) == 4:
        spisok_dr = spisok_pol_dr[1:]
        month_str = spisok_dr[1]
        if month_str in month_list:
            month_ch = str(month_list.index(month_str)+1)
            spisok_dr[1] = month_ch
            dr_dt = pd.to_datetime('.'.join(spisok_dr), dayfirst=True)
        return dr_dt
    return 0

def f_pol_dop(stroka, razdelitel=','):                     # функция выделения пола из ДР
    dr = f_razdelenie_stroki(stroka, razdelitel)[2]
    spisok_pol_dr = dr.split(' ')
    if len(spisok_pol_dr) == 4:
        spisok_pol_dop = spisok_pol_dr[0]
        if spisok_pol_dop == 'родился':
            pol_dop = 'М'
        elif spisok_pol_dop == 'родилась':
            pol_dop = 'Ж'
        else:
            pol_dop = 0
        return pol_dop
    return 0

#display(hh_data_dr)

hh_data_dr['Пол'] = hh_data_dr['Пол, возраст'].apply(f_pol)
hh_data_dr['Возраст'] = hh_data_dr['Пол, возраст'].apply(f_vozrast)
hh_data_dr['ДР'] = hh_data_dr['Пол, возраст'].apply(f_dr)
hh_data_dr['Пол (доп.)'] = hh_data_dr['Пол, возраст'].apply(f_pol_dop)
display(hh_data_dr)
display(hh_data_dr['Пол'].value_counts())
display(hh_data_dr['Пол (доп.)'].value_counts())
display(hh_data_dr['Возраст'].value_counts())
display(hh_data_dr['ДР'].value_counts())


,"Пол, возраст",Пол,Возраст,ДР,Пол (доп.)
0,"Мужчина , 39 лет , родился 27 ноября 1979",мужчина,39,1979-11-27,М
1,"Мужчина , 60 лет , родился 20 марта 1959",мужчина,60,1959-03-20,М
2,"Женщина , 36 лет , родилась 12 августа 1982",женщина,36,1982-08-12,Ж
3,"Мужчина , 38 лет , родился 25 июня 1980",мужчина,38,1980-06-25,М
4,"Женщина , 26 лет , родилась 3 марта 1993",женщина,26,1993-03-03,Ж
...,...,...,...,...,...
44739,"Мужчина , 30 лет , родился 17 января 1989",мужчина,30,1989-01-17,М
44740,"Мужчина , 27 лет , родился 5 марта 1992",мужчина,27,1992-03-05,М
44741,"Женщина , 48 лет , родилась 26 декабря 1970",женщина,48,1970-12-26,Ж
44742,"Мужчина , 24 года , родился 6 октября 1994",мужчина,24,1994-10-06,М


мужчина    36211
женщина     8533
Name: Пол, dtype: int64

М    36211
Ж     8533
Name: Пол (доп.), dtype: int64

30     2834
29     2765
28     2748
31     2675
27     2583
       ... 
15        2
77        1
76        1
14        1
100       1
Name: Возраст, Length: 63, dtype: int64

1986-09-17    20
1988-08-01    19
1990-02-12    19
1993-01-25    18
1988-04-20    18
              ..
1999-04-17     1
1997-11-14     1
1972-11-02     1
1972-10-11     1
1998-10-04     1
Name: ДР, Length: 11081, dtype: int64

### 